In [5]:
import math
import random
import pygame
import sys
import numpy as np
import json
import os
import pandas as pd

pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Simulating Boids Algorithm

Includes code on Boids Algorithm and saving that information over time.

In [6]:
# Size of canvas. These get updated to fill the whole screen.
width = 1000
height = 1000

numBoids = 100
visualRange = 75

boids = []
 
def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
            # 'all_history': []
        })

    df = pd.DataFrame(boids) # Create a DataFrame For Boids
    df.drop(labels='history', axis=1, inplace=True) # Drop the history column
    df['Boids'] = [i for i in range(len(boids))] # Add column 'Boids' for boid number
    return df
def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1

    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005

    centerX = 0
    centerY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors

        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0

    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05

    avgDX = 0
    avgDY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors

        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor
def emo_boid(boid):
    '''
    Here we simply want the flock to disperse; 
    they are not necessarily moving away from any particular object, 
    we just want to break the cohesion (for example, the flock is startled by a loud noise). 
    Thus we actually want to negate part of the influence of the boids rules.
    Of the three rules, it turns out we only want to negate the first one 
    (moving towards the centre of mass of neighbours) -- ie. we want to make the boids move away 
    from the centre of mass. As for the other rules: negating the second rule (avoiding nearby objects) 
    will simply cause the boids to actively run into each other, and negating the third rule 
    (matching velocity with nearby boids) will introduce a semi-chaotic oscillation.
    ... during the course of the simulation, simply make m1 negative to scatter the flock. 
    Setting m1 to a positive value again will cause the flock to spontaneously re-form.
    '''
    
    centeringFactor = 0.005

    centerX = 0
    centerY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors

        boid['dx'] += -1 * (centerX - boid['x']) * centeringFactor
        boid['dy'] += -1 * (centerY - boid['y']) * centeringFactor

def limitSpeed(boid):
    speedLimit = 10

    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, (85, 140, 244), [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)

    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, (85, 140, 244, 102), (int(point[0]), int(point[1])), 1)

# Main animation loop
def animationLoop(animate=False):
    global boids
    num_emo = int(len(boids) * 0.05)
    random_emo_boid = np.random.randint(low=0, high=len(boids), size=(num_emo))
    for i, boid in enumerate(boids):
        if i in random_emo_boid:
            emo_boid(boid)
        else:
            flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

    if animate:
        screen.fill((255, 255, 255))
        for boid in boids:
            drawBoid(screen, boid)

        pygame.display.flip()
        pygame.time.Clock().tick(60)

    df = pd.DataFrame(boids)
    df.drop(labels='history', axis=1, inplace=True)
    df['Boids'] = [i for i in range(len(boids))]
    
    return df

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = False

# initBoids()
# while True:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             pygame.quit()
#             sys.exit()
#     animationLoop(True)

## Running Boids Algorithm and Saving Information Into Pandas DataFrame

In [14]:
directory = '../data/myjson'
if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
import pandas as pd

num_sims = 1
num_time_steps = 10000

# Initialize final_df as an empty DataFrame
final_df = pd.DataFrame()

for n in range(num_sims):  # Run simulation num_sims times
    curr_df_list = []  # List to store DataFrames for current simulation
    curr_df = initBoids()  # Initialize boids with different positions and velocities
    curr_df['Simulation'] = [n] * len(curr_df)  # Set Simulation number
    curr_df['Timestep'] = [0] * len(curr_df)  # Set Timestep number to 0
    curr_df_list.append(curr_df)  # Append initial state to list

    for t in range(1, num_time_steps):  # Run animationLoop() for num_time_steps
        new_df = animationLoop()  # Updates boids to have new positions and velocities
        new_df['Simulation'] = [n] * len(new_df)  # Set Simulation number
        new_df['Timestep'] = [t] * len(new_df)  # Set Timestep number
        curr_df_list.append(new_df)  # Append updated state to list

    # Concatenate all DataFrames in the list once per simulation
    curr_df = pd.concat(curr_df_list, ignore_index=True)
    # Append the result of the current simulation to final_df
    final_df = pd.concat([final_df, curr_df], ignore_index=True)
final_df.reset_index()

,index,x,y,dx,dy,Boids,Simulation,Timestep
0,0,765.977066,207.295667,0.939992,-1.498203,0,0,0
1,1,272.872687,630.655637,-3.594567,4.285805,1,0,0
2,2,664.216733,112.763033,3.938833,3.803304,2,0,0
3,3,955.121396,243.353502,-3.221417,-0.103833,3,0,0
4,4,755.417534,58.766045,1.987104,-4.090977,4,0,0
...,...,...,...,...,...,...,...,...
999995,999995,500.152618,265.870800,-4.906182,8.301848,95,0,9999
999996,999996,338.956249,202.384330,6.292142,2.155068,96,0,9999
999997,999997,198.321050,183.480365,0.013272,5.188505,97,0,9999
999998,999998,325.464019,218.343772,3.935352,9.193095,98,0,9999


In [16]:
final_df

,x,y,dx,dy,Boids,Simulation,Timestep
0,765.977066,207.295667,0.939992,-1.498203,0,0,0
1,272.872687,630.655637,-3.594567,4.285805,1,0,0
2,664.216733,112.763033,3.938833,3.803304,2,0,0
3,955.121396,243.353502,-3.221417,-0.103833,3,0,0
4,755.417534,58.766045,1.987104,-4.090977,4,0,0
...,...,...,...,...,...,...,...
999995,500.152618,265.870800,-4.906182,8.301848,95,0,9999
999996,338.956249,202.384330,6.292142,2.155068,96,0,9999
999997,198.321050,183.480365,0.013272,5.188505,97,0,9999
999998,325.464019,218.343772,3.935352,9.193095,98,0,9999


## Converting Simulation DataFrame to CSV

In [17]:
path_to_save = '../data/simulation.csv'
final_df.to_csv(path_to_save, index=False)

## Save Edges From Each Timestep/Simulation

In [18]:
path_to_save = '../data/simulation.csv'
final_df = pd.read_csv(path_to_save)

### Compute The Edges Per Timestep Per Simulation

In [12]:
from scipy.spatial.distance import pdist, squareform
# Distance threshold

# Function to calculate pairwise distances and return edges
def get_edges(df, threshold):
    
    distances = squareform(pdist(df[['x', 'y']])) # Calculate pairwise distances
    
    close_pairs = distances < threshold # Identify pairs within the threshold distance
    
    # Extract indices of close pairs
    edges = [(i, j) for i in range(len(distances)) for j in range(i+1, len(distances)) if close_pairs[i, j]]
    
    # Create DataFrame for edges
    edges_df = pd.DataFrame(edges, columns=['Boid_i', 'Boid_j'])
    
    return edges_df


edges_dfs = []  # List to collect DataFrames
final_df_groupby = final_df.groupby(['Timestep', 'Simulation'])
for key, item in final_df_groupby:
    edges_df = get_edges(item, visualRange)  # Pass the group directly
    edges_df['Timestep'] = key[0]
    edges_df['Simulation'] = key[1]
    edges_dfs.append(edges_df)

# Concatenate all DataFrames at once, if edges_dfs is not empty
if edges_dfs:
    final_edges_df = pd.concat(edges_dfs, ignore_index=True)
    final_edges_df.reset_index(drop=True, inplace=True)  # Reset index once, outside the loop
else:
    final_edges_df = pd.DataFrame()  # Initialize to an empty DataFrame if no edges were found

final_edges_df

,Boid_i,Boid_j,Timestep,Simulation
0,0,30,0,0
1,0,49,0,0
2,0,69,0,0
3,1,44,0,0
4,1,80,0,0
...,...,...,...,...
10584257,96,97,9999,0
10584258,96,98,9999,0
10584259,96,99,9999,0
10584260,97,98,9999,0


### Save final_edges_df As A CSV

In [13]:
path_to_save = '../data/simulation_edges.csv'
final_edges_df.to_csv(path_to_save, index=False)